В этом блокноте происходит предобработка модели.

Запустим Пайморфи для того, чтобы дальше вытащить из модели ТОЛЬКО существительные.

In [1]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [11]:
from gensim.models import KeyedVectors


Загрузим модель. Использованная модель скачана с [NLPL Word Vectors Repository](http://vectors.nlpl.eu/repository/). Модель (ruwikiruscorpora_upos_skipgram_300_2_2019) обучена на НКРЯ и русской Википедии 2018 года. 

[Ссылка на скачивание модели](http://vectors.nlpl.eu/repository/20/182.zip)

In [ ]:
model = KeyedVectors.load_word2vec_format("model\model.txt", binary=False)

freq_1000.txt -- Файл с 1000-словным списком частотных существительных русского языка (правда, уже не 1000-словный, а подредактированный автором проекта)

In [16]:
freq = open("freq_1000.txt", encoding="utf8")
freq = freq.read().split("\n")

Используем библиотеку Navec (из проекта Наташа для NLP), чтобы проверить, что слова, существующие в модели, реально существуют в русском языке (а не являются, например, опечатками). Модель navec обучена на художественных текстах. 

In [5]:
from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

Выберем из модели model.txt только частотные существительные, существующие также и в navec. 

In [7]:
nouns = [] # все существительые модели

with open ("model.txt", encoding="utf8") as f:
    m = f.readlines()

    for line in m[1:]:
        splitted = line.split(" ")[0].split("_")
        if splitted[1] == "NOUN":
            s = morph.parse(splitted[0])[0].normal_form
            if s in navec:
                if s in freq:
                    if s not in nouns:
                        nouns.append(s)



Запишем частотные существительные в файл freq_nouns.txt, чтобы потом было из чего выбирать

In [28]:
with open ("freq_nouns.txt", "w", encoding="utf-8") as noun_file:
    for n in nouns:
        noun_file.write(f"{n}\n")

Проделаем то же со всеми существительными (уже не отбирая только частотные). Запишем их вектора в файл (не забыв при этом пересчитать мета-данные -- длину модели), а сами существительные -- в список всех существительных.

In [38]:
vec = []
noun_all = []

with open ("model.txt", encoding="utf8") as f:
    m = f.readlines()

    for line in m[1:]:
        splitted = line.split(" ")[0].split("_")
        if splitted[1] == "NOUN":
            s = morph.parse(splitted[0])[0].normal_form
            if s in navec:
                if s not in noun_all:
                        noun_all.append(s)
                        vec.append(line.replace("_NOUN", "").replace(splitted[0], s))

    with open ("noun_model.txt", "w", encoding="utf8") as f: # новая модель, содержащая только существительные
        f.write(f"{len(vec)} 300\n")
        f.write("".join(vec))


Запишем все существительные в файл.

In [ ]:
with open ("all_nouns.txt", "w", encoding="utf-8") as all_noun_file:
    for n in noun_all:
        all_noun_file.write(f"{n}\n")

Итак, у нас получилось 3 файла: 

`all_nouns.txt` -- все существительные, содержащиеся в модели

`freq_nouns.txt` -- частотные существительные, содержащиеся в модели. Из этих существительных будет происходить выбор слов для угадывания (чтобы не пришлось угадывать вещи типа "диффузия + электрорадиатор")

`noun_model.txt` -- векторная модель, содержащая только существительные